In [160]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, auc
import pandas as pd
import numpy as np
from keras.datasets import boston_housing
from keras.datasets import reuters
import numpy as np
import os
import tensorflow as tf
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
import matplotlib.pyplot as plt
import time
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import *
from sklearn.neighbors import *
from sklearn.datasets import make_classification
import xgboost as xgb
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
import warnings

In [233]:
warnings.filterwarnings('ignore')
df = pd.read_excel(r'C:\Users\Big data\Desktop\class\funcardproject\斷詞與和卡額度_20群.xls',encoding='utf-16')
df = df.loc[:, ["age","serveTime","credLimit","Loan","SalPerY","holdCard","Career","credLimit_group"]] 
df =df.dropna(
    axis=0,     # 0: 对行进行操作; 1: 对列进行操作
    how='any'   # 'any': 只要存在 NaN 就 drop 掉; 'all': 必须全部是 NaN 才 drop 
    )
df["credLimit_group"] = df["credLimit_group"].astype(int)
print(df.shape)
print(df.info())

(1680, 8)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1680 entries, 0 to 1686
Data columns (total 8 columns):
age                1680 non-null float64
serveTime          1680 non-null float64
credLimit          1680 non-null int64
Loan               1680 non-null float64
SalPerY            1680 non-null int64
holdCard           1680 non-null int64
Career             1680 non-null int64
credLimit_group    1680 non-null int32
dtypes: float64(3), int32(1), int64(4)
memory usage: 111.6 KB
None


# <font color=#FF8800 size=20 face="標楷體"> 定義 </font>

In [203]:
def cut(random):
    # 切分訓練,測試
    train_data, test_data, train_targets, test_targets = train_test_split(df.loc[:, ["age","serveTime","Loan","SalPerY","holdCard","Career"]] , df.loc[:, ["credLimit_group"]] , test_size=0.3,random_state=random)
    # 標準化
    mean = train_data.mean(axis=0)
    train_data -=mean
    std = train_data.std(axis=0)
    train_data/=std
    test_data-=mean
    test_data/=std
    return train_data, test_data, train_targets, test_targets

In [281]:
def model_test(train_data, test_data, train_targets, test_targets,model):
    model.fit(train_data, train_targets)
    訓練準確度 = model.score(train_data, train_targets)
    驗證準確度 = model.score(test_data, test_targets)
    print(model,"模型")
    print("訓練準確度:",訓練準確度,"%")
    print("訓練準確度:",驗證準確度,"%")
    print("___________________________________________________________________________________________________________")
    

In [197]:
#因樣本不足,為避免資訊洩漏,與建立相同比較基準使用默認參數
#若要調參,應對
model_xgb = xgb.XGBClassifier()
model_GBM = GradientBoostingClassifier()
model_AdaBoost = AdaBoostClassifier()
model_Bagging = BaggingClassifier()
model_ExtraTreesClassifier = ExtraTreesClassifier()
model_SVM = SVC()
model_KNN = KNeighborsClassifier()
model_LDA = LinearDiscriminantAnalysis()
model_rf = RandomForestClassifier()
model_Gaussian = GaussianProcessClassifier()
model_BernoulliNB = BernoulliNB()
model_DecisionTreeClassifier = DecisionTreeClassifier()
model_ExtraTreeClassifier = ExtraTreeClassifier()
model_MultiOutputClassifier = MultiOutputClassifier(SVC())#不能投票,可能是*粉
model_OneVsOneClassifier = OneVsOneClassifier(SVC())
model_OneVsRestClassifier = OneVsRestClassifier(SVC())#不能投票,可能是*粉
model_OutputCodeClassifier = OutputCodeClassifier(SVC())
#清單
modellist=[model_xgb,
           model_GBM,
           model_AdaBoost,
           model_Bagging,
           model_ExtraTreesClassifier,
           model_SVM,
           model_KNN,
           model_LDA,
           model_rf,
           model_Gaussian,
           model_BernoulliNB,
           model_DecisionTreeClassifier,
           model_ExtraTreeClassifier,
           model_MultiOutputClassifier,
           model_OneVsOneClassifier,
           model_OneVsRestClassifier,
           model_OutputCodeClassifier]

In [277]:
#model_MultiOutputClassifier,('model_OneVsRestClassifier' ),不能用
#選幾個比較強的模型
#可逐一剔除,觀察準確度變化
def vote(train_data, test_data, train_targets, test_targets,i):
    Voting = VotingClassifier(estimators=[('model_xgb', model_xgb),#good
                                          ('model_GBM', model_GBM),#good
#                                           ('model_AdaBoost', model_AdaBoost),
                                          ('model_ExtraTreesClassifier', model_ExtraTreesClassifier),
                                          ('model_SVM', model_SVM),
                                          ('model_KNN', model_KNN),#poor
#                                           ('model_LDA', model_LDA),
#                                           ('model_rf', model_rf),
                                          ('model_Gaussian', model_Gaussian),#poor
#                                           ('model_BernoulliNB', model_BernoulliNB),#真的白努力
#                                           ('model_DecisionTreeClassifier', model_DecisionTreeClassifier),
                                          ('model_ExtraTreeClassifier', model_ExtraTreeClassifier),
#                                           ('model_OneVsOneClassifier', model_OneVsOneClassifier),
#                                           ('model_OutputCodeClassifier', model_OutputCodeClassifier)
                                         ])
    Voting.fit(train_data, train_targets)
    訓練準確度 = Voting.score(train_data, train_targets)
    驗證準確度 = Voting.score(test_data, test_targets)
    ㄎㄎ = print("第",i,"個seed訓練準確度:",訓練準確度,"%")
    ㄏㄏ = print("第",i,"個seed測試準確度:",驗證準確度,"%")
    return ㄎㄎ,ㄏㄏ

# <font color=#FF8800 size=20 face="標楷體"> 執行 </font>

In [282]:
#每個模型分別的準度,觀察which模型比較強
[(cut(4),model_test(train_data, test_data, train_targets, test_targets,model))for model in modellist]

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1) 模型
訓練準確度: 0.6292517006802721 %
訓練準確度: 0.376984126984127 %
___________________________________________________________________________________________________________
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
           

OutputCodeClassifier(code_size=1.5,
                     estimator=SVC(C=1.0, cache_size=200, class_weight=None,
                                   coef0=0.0, decision_function_shape='ovr',
                                   degree=3, gamma='auto_deprecated',
                                   kernel='rbf', max_iter=-1, probability=False,
                                   random_state=None, shrinking=True, tol=0.001,
                                   verbose=False),
                     n_jobs=None, random_state=None) 模型
訓練準確度: 0.40901360544217685 %
訓練準確度: 0.3253968253968254 %
___________________________________________________________________________________________________________


[((           age  serveTime      Loan   SalPerY  holdCard    Career
   1408 -0.219644  -0.597258 -0.558358  0.147288  0.835959 -0.204124
   167  -1.088456  -0.229708 -0.558358 -0.428294 -1.195214 -0.204124
   19   -0.436847  -0.229708 -0.558358 -0.572190  0.835959 -1.837117
   391  -0.436847   0.137843 -0.558358 -0.322771  0.835959  3.061862
   923  -0.436847  -0.229708 -0.558358 -0.140503 -1.195214 -0.204124
   450   0.431965  -0.556419 -0.558358 -0.284399  0.835959 -0.204124
   332  -0.515481  -0.576838  0.684506 -0.351550 -1.195214 -0.204124
   526  -0.162680   0.137843 -0.558358 -0.284399  0.835959 -0.204124
   44   -1.305659  -0.447719 -0.558358 -0.553004 -1.195214 -1.837117
   1129  1.735182   0.382876 -0.558358 -0.092538  0.835959  1.428869
   1220  0.866370   0.176368 -0.558358  0.003393 -1.195214 -0.204124
   730  -0.002441  -0.515580 -0.558358 -0.217247  0.835959 -0.204124
   1515  1.952385  -0.576838  0.419312  0.291184 -1.195214 -0.204124
   574  -0.219644  -0.535999 -0.55

In [278]:
#投票模型的準度
[(cut(i),vote(train_data, test_data, train_targets, test_targets,i)) for i in range(10)]
#可以看到不同的樣本選取下,準確度與變異性皆優於單個模型

第 0 個seed訓練準確度: 0.7729591836734694 %
第 0 個seed測試準確度: 0.40674603174603174 %
第 1 個seed訓練準確度: 0.7721088435374149 %
第 1 個seed測試準確度: 0.39285714285714285 %
第 2 個seed訓練準確度: 0.7755102040816326 %
第 2 個seed測試準確度: 0.3968253968253968 %
第 3 個seed訓練準確度: 0.7721088435374149 %
第 3 個seed測試準確度: 0.3968253968253968 %
第 4 個seed訓練準確度: 0.7721088435374149 %
第 4 個seed測試準確度: 0.4146825396825397 %
第 5 個seed訓練準確度: 0.7729591836734694 %
第 5 個seed測試準確度: 0.39880952380952384 %
第 6 個seed訓練準確度: 0.7721088435374149 %
第 6 個seed測試準確度: 0.4146825396825397 %
第 7 個seed訓練準確度: 0.7729591836734694 %
第 7 個seed測試準確度: 0.40476190476190477 %
第 8 個seed訓練準確度: 0.7721088435374149 %
第 8 個seed測試準確度: 0.4007936507936508 %
第 9 個seed訓練準確度: 0.7721088435374149 %
第 9 個seed測試準確度: 0.40476190476190477 %


[((           age  serveTime      Loan   SalPerY  holdCard    Career
   1406  1.302321   1.084319  2.023564  0.118474  0.844795 -0.191153
   77   -1.504409  -0.117919 -0.564954 -0.522811  0.844795 -0.191153
   1651  1.518223  -0.358366 -0.564954  3.004255 -1.182713  2.974988
   39   -0.424898  -0.478590 -0.564954 -0.550294 -1.182713 -0.191153
   1105  0.222809   0.002305 -0.564954 -0.110556 -1.182713 -0.191153
   944  -0.424898  -0.578776  0.729305 -0.156362 -1.182713 -0.191153
   884  -0.424898  -0.017732 -0.564954 -0.202168  0.844795 -0.191153
   678   0.654614   0.234740 -0.564954 -0.247974  0.844795 -0.191153
   1288 -0.208995  -0.338329 -0.002276 -0.000622 -1.182713  2.974988
   1005 -0.640800  -0.338329 -0.564954 -0.110556  0.844795 -0.191153
   875  -0.208995  -0.518664 -0.007083 -0.202168  0.844795 -0.191153
   724   0.027772   0.132667  0.058600 -0.247974  0.844795 -0.191153
   831   0.006907  -0.578776 -0.564954 -0.202168 -1.182713 -0.191153
   114  -0.755218  -0.327941 -0.56